<a href="https://colab.research.google.com/github/hafeezullah05/Roads_PathHole_Detection/blob/main/DSProject_Raoad_PathHoleDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preparation

## DS1 Label transformation
This part deals with the transformation of the labels into the YOLOv5 format.

### Imports

In [ ]:
from PIL import Image
import os
from shutil import copyfile

### Read Test Data Labels

In [ ]:
# Saves Lines in list

input_labels_test = "DS1\simpleTestFullSizeAllPotholesSortedFullAnnotation.txt"
with open(input_labels_test) as file:
    lines = []
    for line in file:
        lines.append(line.strip().replace("  ",""))

file.close()

# Example ["Test data\G0011476.bmp 2 2176 1580 242 44 1726 1458 106 26 "]
#lines[0]

### Create New Labels for Test Data

In [ ]:
# for Test data
error_log_test = open("ErrorLogTest.txt","w+")
imgLabelNameTest = []

for line in lines:

 # Gain Information 
    line_list = line.split()
    name = line_list[1].replace("data\\"," ").replace(".bmp", "").strip()
    num_potholes = int(line_list[2])
    
    image_path = "DS1\TestData\\" + name + ".JPG"
    try:
        im = Image.open(image_path)
        img_width, img_height = im.size
    
      # gain information for individual potholes
        potholes = []
        for n in range(0,num_potholes):
        #x_center, y_center, width, height
            potholes.append([(int(line_list[3+4*n]) + int(line_list[5+4*n])/2) / img_width,
                             (int(line_list[4+4*n]) + int(line_list[6+4*n])/2) / img_height,
                             int(line_list[4+4*n]) / img_width,
                             int(line_list[5+4*n]) / img_height])
    
      #Write new file
        path = "DS1_new\labels\Test"
        new_file = open(path + "\\" + name+ ".txt", "w+")
        for n in range (0, num_potholes): 
            new_file.write("0 %f %f %f %f\n" %(potholes[n][0], potholes[n][1], potholes[n][2], potholes[n][3]))
        new_file.close()
        imgLabelNameTest.append(name)
    except:
        error_log_test.write("%s resulted in error\n" %name)
error_log_test.close()

### Read Training Data Labels

In [ ]:
# Saves Lines in list

input_labels_test = "DS1\simpleTrainFullPhotosSortedFullAnnotations.txt"
with open(input_labels_test) as file:
    lines = []
    for line in file:
        lines.append(line.strip().replace("  ",""))

file.close()

# Example: "Train data\Positive data\G0010110.bmp 2 2002 1370 58 18 1896 1342 50 16"
lines[0]

'Train data\\Positive data\\G0010033.bmp 6 1990 1406 66 14 1464 1442 92 16 1108 1450 54 16 558 1434 102 16 338 1450 72 18 262 1450 58 22'

### Create New Labels for Training Data
(Note: only images with potholes are considered)

In [ ]:
# for Train data
error_log_train = open("ErrorLogTrain.txt","w+")
imgLabelNameTrain = []


for line in lines:

 # Gain Information 
    line_list = line.split()
    name = line_list[2].replace("data\\"," ").replace(".bmp", "").strip()
    num_potholes = int(line_list[3])
            
    image_path = "DS1\TrainData\PositiveData\\" + name + ".JPG"
    try:
        im = Image.open(image_path)
        img_width, img_height = im.size
    
      # gain information for individual potholes and change format
        potholes = []
        for n in range(0,num_potholes):
        #x_center, y_center, width, height
            potholes.append([(int(line_list[4+4*n]) + int(line_list[6+4*n])/2) / img_width,
                             (int(line_list[5+4*n]) + int(line_list[7+4*n])/2) / img_height,
                             int(line_list[5+4*n]) / img_width,
                             int(line_list[6+4*n]) / img_height])
    
      #Write new file
        path = "DS1_new\labels\Train"
        new_file = open(path + "\\" + name + ".txt", "w+")
        for n in range (0, num_potholes): 
            new_file.write("0 %f %f %f %f\n" %(potholes[n][0], potholes[n][1], potholes[n][2], potholes[n][3]))
        new_file.close()
        imgLabelNameTrain.append(name)

    except:
        error_log_train.write("%s resulted in error\n" %name)
        
error_log_train.close()

### Copy the right pictures into new files

In [ ]:
# Commented to avoid long runtimes
"""
for name in imgLabelNameTest:
    oldPath = "DS1\TestData\\" + name +".JPG"
    newPath = "DS1_new\images\TestData\\" + name + ".JPG"
    copyfile(oldPath, newPath)
    
for name in imgLabelNameTrain:
    oldPath = "DS1\TrainData\PositiveData\\" + name +".JPG"
    newPath = "DS1_new\images\TrainData\\" + name + ".JPG"
    copyfile(oldPath, newPath)
"""

'\nfor name in imgLabelNameTest:\n    oldPath = "DS1\\TestData\\" + name +".JPG"\n    newPath = "DS1_new\\images\\TestData\\" + name + ".JPG"\n    copyfile(oldPath, newPath)\n    \nfor name in imgLabelNameTrain:\n    oldPath = "DS1\\TrainData\\PositiveData\\" + name +".JPG"\n    newPath = "DS1_new\\images\\TrainData\\" + name + ".JPG"\n    copyfile(oldPath, newPath)\n'

In [ ]:
convertedLabels = len(imgLabelNameTest)+len(imgLabelNameTrain)
print("Total images&labels converted to YOLO format: %d" %convertedLabels)

Total images&labels converted to YOLO format: 2030


## DS2 Removal of Dublicates

In [ ]:
# create a list of images without dublicates (training data)

path = "DS2\\training\\images"
files = os.listdir(path)

pictureNumber=[]
for f in files:
    pictureNumber.append(int(f.replace("_png","").replace("potholes","").split(".")[0]))

imagesWithoutDublicates=[]
imagesWithoutDublicates.append(files[0])  # necessary because switches 
for i in range(1,len(pictureNumber)):
    if  pictureNumber[i] != pictureNumber[i-1]:
        imagesWithoutDublicates.append(files[i])

In [ ]:
# Copy all chosen images into new folder
"""
for name in imagesWithoutDublicates:
    oldPath = "DS2\training\images\\"+ name
    newPath = "DS2_new\training\images\\"+ name
    copyfile(oldPath, newPath)
"""

'\nfor name in imagesWithoutDublicates:\n    oldPath = "DS2\training\\images\\"+ name\n    newPath = "DS2_new\training\\images\\"+ name\n    copyfile(oldPath, newPath)\n'

In [ ]:
# Copy the labels for the chosen images
"""
for name in imagesWithoutDublicates:
    oldPath = "DS2\training\labels\\"+ name.replace(".jpg", ".txt")
    newPath = "DS2_new\training\labels\\"+ name.replace(".jpg", ".txt")
    copyfile(oldPath, newPath)
"""

'\nfor name in imagesWithoutDublicates:\n    oldPath = "DS2\training\\labels\\"+ name.replace(".jpg", ".txt")\n    newPath = "DS2_new\training\\labels\\"+ name.replace(".jpg", ".txt")\n    copyfile(oldPath, newPath)\n'

In [ ]:
# Copy test data (no dublicates)
"""
files = os.listdir("DS2\test\images")
    for file in files:
        oldPath =  path + "\\" + file
        newPath = "DS2_new\test\images\\" + file
        copyfile(oldPath, newPath)
        
files = os.listdir("DS2\test\labels")
    for file in files:
        oldPath =  path + "\\" + file
        newPath = "DS2_new\test\labels\\" + file
        copyfile(oldPath, newPath)
"""

'\nfiles = os.listdir("DS2\test\\images")\n    for file in files:\n        oldPath =  path + "\\" + file\n        newPath = "DS2_new\test\\images\\" + file\n        copyfile(oldPath, newPath)\n        \nfiles = os.listdir("DS2\test\\labels")\n    for file in files:\n        oldPath =  path + "\\" + file\n        newPath = "DS2_new\test\\labels\\" + file\n        copyfile(oldPath, newPath)\n'

## Merge all Data together

In [ ]:
# Create final dataset (without train-test split)

#  Get images:
path1 = "DS1_new\images\TestData"
path2 = "DS1_new\images\TrainData"
path3 = "DS2_new\training\images"
path4 = "DS2_new\test\images"

#get labels
path5 = "DS1_new\labels\Test"
path6 = "DS1_new\labels\Train"
path7 = "DS2_new\training\labels"
path8 = "DS2_new\test\labels"

imagepath = [path1, path2, path3, path4]
labelpath = [path5, path6, path7, path8]

"""
for path in imagepath:
    files = os.listdir(path)
    for file in files:
        oldPath =  path + "\\" + file
        newPath = "Dataset_final\images\\" + file
        copyfile(oldPath, newPath)
      
for path in labelpath:
    files = os.listdir(path)
    for file in files:
        oldPath =  path + "\\" + file
        newPath = "Dataset_final\labels\\" + file
        copyfile(oldPath, newPath)
"""

'\nfor path in imagepath:\n    files = os.listdir(path)\n    for file in files:\n        oldPath =  path + "\\" + file\n        newPath = "Dataset_final\\images\\" + file\n        copyfile(oldPath, newPath)\n      \nfor path in labelpath:\n    files = os.listdir(path)\n    for file in files:\n        oldPath =  path + "\\" + file\n        newPath = "Dataset_final\\labels\\" + file\n        copyfile(oldPath, newPath)\n'